In [2]:
import numpy as np
from panda import Panda
from ik_ca import limit_joints
from ik_ca import ik_ca, ik_ca_neg
from ik_ca2 import ik_ca as ik_ca2
from ik_ca2 import ik_ca_neg as ik_ca_neg2

In [76]:
rob = Panda()
q = np.random.randn(7)
# 将随机生成的角度限制到franka的关节极限内
q = limit_joints(q)

In [88]:
# 先令2角为正
q[1] = np.abs(q[1])

In [89]:
# fk不挂载tool,如果挂载,乘一个常值矩阵换算到flange即可
T = rob.forward_flange(q)

In [90]:
iks = [ik_ca, ik_ca_neg, ik_ca2, ik_ca_neg2]

In [91]:
# 验证，在分支1或3
for ik in iks:
    qi = ik(T, q[-1])
    qi = limit_joints(qi)
    Ti = rob.forward_flange(qi)
    ik_valid = np.allclose(Ti, T)
    on_this_branch = np.allclose(qi, q)
    print('反解有效：{}, 分支正确：{}'.format(ik_valid, on_this_branch))

反解有效：True, 分支正确：True
反解有效：True, 分支正确：False
反解有效：True, 分支正确：False
反解有效：True, 分支正确：False


In [92]:
# 再令2角为负
q[1] = -np.abs(q[1])
T = rob.forward_flange(q)

In [93]:
# 验证，在分支2或4
for ik in iks:
    qi = ik(T, q[-1])
    qi = limit_joints(qi)
    Ti = rob.forward_flange(qi)
    ik_valid = np.allclose(Ti, T)
    on_this_branch = np.allclose(qi, q)
    print('反解有效：{}, 分支正确：{}'.format(ik_valid, on_this_branch))

反解有效：True, 分支正确：False
反解有效：True, 分支正确：True
反解有效：True, 分支正确：False
反解有效：True, 分支正确：False


In [94]:
# %timeit rob.forward_flange(q)

In [95]:
# %timeit ik(T, q[-1])